## NOTEBOOK IS NOW DONE

This notebook is now done... 
The database is built, and most queries can be done using SQL language, so I make a new notebook for examples.

In [1]:
cd ..

C:\Users\csav6865\Documents\GitHub\picarro-calculator


In [2]:
import Picarro as pica
import PicarroDatabase as pdb
import pickle

In [3]:
files = pdb.checkforrawdata(r"J:\c715\Picarro\Results\Results 2019\Raw data")

There is no new raw data in directory: J:\c715\Picarro\Results\Results 2019\Raw data


In [4]:
conn = pdb.CreateConnection(r"J:\c715\Picarro\Results\Database\data.db")

for file in files:
    pdb.AddRaw(file,conn)
    try:
        pdb.AddSummaryRun(file,conn)
    except:
        print("An error occurred")

TypeError: 'NoneType' object is not iterable

In [5]:
import numpy as np
import glob
import pandas as pd

Here, we've selected the run ids and their respective nicknames for the year 2019

In [ ]:
df = pd.read_sql_query("select * from runs r where r.'Identifier 2' = 'TP';",conn)
df

In [ ]:
df = pd.read_sql_query("select * from rawrun rw where rw.'Identifier 2'='YD';",conn)
df

we can also select the runs based on what the name is like, for example: guffert runs.

In [ ]:
### In case the database is rebuilt, and the run look up table needs to be done again, uncomment the next bit of code.


#strips = ["~$","J:\\c715\\Picarro\\Results\\Results 2019\\Picarro-Results-",".xlsx","-","_"]

#nicknames = []

#for i in glob.glob(r"J:\c715\Picarro\Results\Results 2019\*.xlsx"):
#    full = i
#    for j in strips:

#        full = full.replace(j," ")
#    #print(full)
#    if full.startswith(" 2019") == True:
#        nicknames.append(full)
    
#RUN_IDS = glob.glob(r"J:\c715\Picarro\Results\Results 2019\Raw data\*.csv")

#for i,j in zip(nicknames,RUN_IDS):
#    pdb.AddRun(j,i,conn)

In [ ]:
df = pd.read_sql_query("select * from runlookup rlk where rlk.NickName like '%Hundsalm%';",conn)
df

Now let's execute a statement where the NickName is replaced.

In [12]:
import sqlite3
from sqlite3 import Error

def ReplaceName(conn,RUN_ID,newname):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param RUN_ID: the run id, an eight digit integer with format yyyymmdd
    :return: updated table name in 

    https://www.sqlitetutorial.net/sqlite-python/create-tables/
    """
    
    statement= """UPDATE runlookup
    SET RUN_ID = {0}, NickName = '{1}'
    WHERE RUN_ID = {0};""".format(RUN_ID,newname)
    
    try:
        c = conn.cursor()

        c.execute(statement)
    except Error as e:
            print(e)
    conn.commit()

In [13]:
ids = [20190513,20190514,20190515,20190516,20190707]
newnames=['2019 05 14 TR Hundsalm Run01',
         '2019 05 15 TR Hundsalm Run02',
         '2019 05 16 TR Hundsalm Run03',
         '2019 05 17 TR Hundsalm Run04',
         '2019 07 08 TR Hundsalm Run05']
for i,j in zip(ids,newnames):
    ReplaceName(conn,i,j)
    



In [14]:
df = pd.read_sql_query("select * from runlookup rlk where rlk.NickName like '%Hundsalm%';",conn)
df

,NickName,RUN_ID
0,2019 05 14 TR Hundsalm Run01,20190513
1,2019 05 15 TR Hundsalm Run02,20190514
2,2019 05 16 TR Hundsalm Run03,20190515
3,2019 05 17 TR Hundsalm Run04,20190516
4,2019 07 08 TR Hundsalm Run05,20190707


In [16]:
statement ="""SELECT * from runlookup rlk,runs r 
            WHERE (
            rlk.NickName like '%Hundsalm%'
            AND r.RUN_ID= rlk.RUN_ID
            );"""

df = pd.read_sql_query(statement,conn)
df

,NickName,RUN_ID,key,Identifier 1,Identifier 2,RUN_ID,position,d18O vsmow,d18O stdev. vsmow,d18O counts,d2H vsmow,d2H stdev. vsmow,d2H counts,inside GMWL
0,2019 05 15 TR Hundsalm Run02,20190514,9635,TAP,_Conditioning,20190514,1,-13.391099,0.017152,4,-93.598271,0.305457,4,inside
1,2019 05 15 TR Hundsalm Run02,20190514,9636,HAUS1,_Standard,20190514,2,0.622775,0.089097,10,3.308604,0.352785,10,outside
2,2019 05 15 TR Hundsalm Run02,20190514,9637,HAUS2,_Standard,20190514,3,-29.859888,0.017355,10,-230.080662,0.017337,10,inside
3,2019 05 15 TR Hundsalm Run02,20190514,9638,TAP,_Standard,20190514,4,-13.442887,0.051945,10,-94.527942,0.388874,10,inside
4,2019 05 15 TR Hundsalm Run02,20190514,9639,W22,_Control W22,20190514,5,-3.246392,0.054727,4,-20.369468,0.560725,4,inside
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,2019 07 08 TR Hundsalm Run05,20190707,9900,HU-042-7DAY,20190601Hundsalm,20190707,23,-12.962829,0.019094,4,-95.576305,0.234611,4,inside
77,2019 07 08 TR Hundsalm Run05,20190707,9901,HU-043-7DAY,20190601Hundsalm,20190707,24,-12.736365,0.035587,4,-91.859017,0.197248,4,inside
78,2019 07 08 TR Hundsalm Run05,20190707,9902,XX4-7DAY,20190601Hundsalm,20190707,25,-9.190924,0.080785,4,-64.388225,0.472473,4,inside
79,2019 07 08 TR Hundsalm Run05,20190707,9903,XX5-7DAY,20190601Hundsalm,20190707,26,-10.149943,0.065672,2,-72.017654,0.443688,3,inside


In [ ]:
df = pd.read_sql_query("select r.'Identifier 1',r.'Identifier 2' from rawrun r where RUN_ID = 20190513",conn)

In [ ]:
df1 = pd.read_csv(r"J:\c715\Picarro\Results\Results 2019\Raw data\HKDS2009_IsoWater_20190513_142143.csv")

In [ ]:
df1.head(50)

In [ ]:
pd.read_sql_query("select * from runlookup rlk,runs r where (rlk.NickName like '%Hundsalm%' and (r.RUN_ID = rlk.RUN_ID and r.'Identifier 2'  like '%Hundsalm%'));",conn)

In [ ]:
pd.read_sql_query("select * from runs where")

In [ ]:
pd.read_sql_query("select * from runs",conn)